In [4]:
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

client = OpenAI()

## Upload file for retrieval

In [7]:
file = client.files.create(
  file=open("zia_profile.pdf", "rb"),
  purpose="assistants"
)
print(file)

# Attach file to assistant
assistant = client.beta.assistants.create(
  instructions="You are a customer support chatbot. Use your knowledge base to best respond to customer queries.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}], # enable retrieval tool
  file_ids=[file.id] # attach file id
)

FileObject(id='file-ghDJJcvDBXSoBMcZTtKSNp6Q', bytes=48802, created_at=1703437236, filename='zia_profile.pdf', object='file', purpose='assistants', status='processed', status_details=None)


## Thread and Message

In [8]:
# Create a thread
thread = client.beta.threads.create()

# Attach a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the date of birth of Zia?"
)

## Run

In [9]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

## Response

In [11]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(messages)

print("")
# displaying in conversation style
for message in reversed(messages.data):
  # revesed() because assistant message was appended and it is listed first.
  print(message.role + ": " + message.content[0].text.value)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_TlGkswtTNxnGlxCPIGmsE36W', assistant_id='asst_NdsE0SxIee9YDcgVeTBhXdIq', content=[MessageContentText(text=Text(annotations=[TextAnnotationFileCitation(end_index=46, file_citation=TextAnnotationFileCitationFileCitation(file_id='file-ghDJJcvDBXSoBMcZTtKSNp6Q', quote='Zia Khan was born in Sialkot in 1961 in an army garrison'), start_index=36, text='【7†source】', type='file_citation')], value='Zia Khan was born in Sialkot in 1961【7†source】. Therefore, his date of birth is in 1961.'), type='text')], created_at=1703437243, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_ACi0E299WEK7Kc7yJqKdqx0t', thread_id='thread_9k9jLJ28qhLxfbblKLIvusxM'), ThreadMessage(id='msg_k7XAthp5NleEsl9UIvEDpBeZ', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='What is the date of birth of Zia?'), type='text')], created_at=1703437239, file_ids=[], metadata={}, object='thread.message', role='user'

## Deleting files

In [ ]:
file_deletion_status = client.beta.assistants.files.delete(
    assistant_id=assistant.id,
    file_id=file.id
)

# Detaching the file from the assistant removes the file from the retrieval index and means you will no longer be charged for the storage of the indexed file.